In [ ]:
import pandas as pd

test_file = 'problematic_samples/070723-zip/images-output/07072023-2/delivery/CUR6470-01_2438_05022023_0001126908/text_formatted.txt'


formatted_output = pd.read_csv(test_file, sep='|')
formatted_output.head()



In [ ]:
import re

# We take the regular expressions from the lambda function extracting the text
approval_re = r"[(A-Z0-9_]+[_ ].*?(?:MATERIALS?|MODIFIED|MODIFIED_2023|ACCEPTED|SPN|)\d{7,8}|[A-Z0-9]+_[0-9A-Z]+_[A-Z0-9]+_[A-Z]+"
twenty_re = (
    r"[A-Z][A-Z][OSC0-9][A-Z][A-Z][A-Z][A-Z0-9][A-Z0-9][A-Z0-9]+[ _.][A-Z\d]{3,4}$"
)
field_regular_expressions = {
    "ZIP_CODE_4": "^[0-9]{5}(?:-[0-9]{4})?$",
    "STATE": r"^[A-Z]{2}$",
    "REGULATORY_APPROVAL_ID": rf"^({approval_re}) ({twenty_re})$",
}

# We compile all the regular expression in the above dictionary
# overwriting the values
for column in field_regular_expressions:
    field_regular_expressions[column] = re.compile(
        field_regular_expressions[column]
    )

In [ ]:
from functools import reduce

# Add f"{column}_check" columns to the formatted_output dataframe indicating if 
# a regular expression is violated by the column in a particular row
for column in field_regular_expressions:
    formatted_output[f"{column}_check"] = formatted_output[column].apply(
        lambda val: True if field_regular_expressions[column].findall(val) else False
    )

In [ ]:


# Filter only those row that have at least one violation of a regular expression in
# a column.
filtered_formatted_output = formatted_output[
    reduce(
        lambda acc, key: acc | (formatted_output[f'{key}_check'] == False),
        field_regular_expressions.keys(),
        False,
    )
]


# Delete all the values in columns we are checking that weren't violating a regular expression in the 
# resulting filtered dataframe.
for column in field_regular_expressions:
    filtered_formatted_output[f"{column}"] = filtered_formatted_output[[f"{column}_check", column]].apply(lambda row:
        row[column] if not row[f"{column}_check"] else None,
        axis = 1,                                                        
    )
  

filtered_formatted_output.head()

In [4]:
import pandas as pd
import re
from functools import reduce

def get_text_formatted_errors(txt_file_in, txt_file_out):
    formatted_output = pd.read_csv(txt_file_in, sep='|')
    
    # We take the regular expressions from the lambda function extracting the text
    approval_re = r"[(A-Z0-9_]+[_ ].*?(?:MATERIALS?|MODIFIED|MODIFIED_2023|ACCEPTED|SPN|)\d{7,8}|[A-Z0-9]+_[0-9A-Z]+_[A-Z0-9]+_[A-Z]+"
    twenty_re = (
        r"[A-Z][A-Z][OSC0-9][A-Z][A-Z][A-Z][A-Z0-9][A-Z0-9][A-Z0-9]+[ _.][A-Z\d]{3,4}$"
    )
    field_regular_expressions = {
        "ZIP_CODE_4": "^[0-9]{5}(?:-[0-9]{4})?$",
        "STATE": r"^[A-Z]{2}$",
        "REGULATORY_APPROVAL_ID": rf"^({approval_re}) ({twenty_re})$",
    }

    # We compile all the regular expression in the above dictionary
    # overwriting the values
    for column in field_regular_expressions:
        field_regular_expressions[column] = re.compile(
            field_regular_expressions[column]
    )
        
    # Add f"{column}_check" columns to the formatted_output dataframe indicating if 
    # a regular expression is violated by the column in a particular row
    for column in field_regular_expressions:
        formatted_output[f"{column}_check"] = formatted_output[column].apply(
            lambda val: True if field_regular_expressions[column].findall(val) else False
        )

    # Filter only those row that have at least one violation of a regular expression in
    # a column.
    filtered_formatted_output = formatted_output[
        reduce(
            lambda acc, key: acc | (formatted_output[f'{key}_check'] == False),
            field_regular_expressions.keys(),
            False,
        )
    ]

    # Delete all the values in columns we are checking that weren't violating a regular expression in the 
    # resulting filtered dataframe.
    for column in field_regular_expressions:
        filtered_formatted_output[f"{column}"] = filtered_formatted_output[[f"{column}_check", column]].apply(lambda row:
            row[column] if not row[f"{column}_check"] else None,
            axis = 1,                                                        
        )
  
    filtered_formatted_output = filtered_formatted_output[list(filtered_formatted_output.keys())]


    # We only save if there is a record
    if filtered_formatted_output.shape[0] > 0:
        print(f'Saving: {txt_file_out}')
        filtered_formatted_output.to_csv(txt_file_out, sep='|')
    

In [5]:
from pathlib import Path

paths = []
for path in Path('problematic_samples/070723-zip/images-output').rglob('*.txt'):
    paths.append((str(path), f'{str(path)[:-4]}_err.txt'))
paths

for path_in, path_out in paths:
    get_text_formatted_errors(path_in, path_out)

TypeError: expected string or bytes-like object